# Dataset Loading

In [1]:
import os

base_path = "/kaggle/input"

print("Available datasets in /kaggle/input:")
print(os.listdir(base_path))

# Replace with your actual dataset folder name
data_path = "/kaggle/input/sketches-png-dataset"

# Show top-level contents of the folder
print("Files/Folders in dataset directory:")
print(os.listdir(data_path))

print(os.listdir("/kaggle/input/sketches-png-dataset"))

data_path = "/kaggle/input/sketches-png-dataset"

# List first-level folders (object categories?)
print("Top-level folders:")
print(os.listdir(data_path))

# Look inside one of the folders (e.g., apple)
sample_class = os.listdir(data_path)[0]
print(f"\nFiles inside '{sample_class}':")
print(os.listdir(os.path.join(data_path, sample_class))[:5])  # show first 5 images

Available datasets in /kaggle/input:
['sketch-features', 'sketches-png-dataset']
Files/Folders in dataset directory:
['png']
['png']
Top-level folders:
['png']

Files inside 'png':
['fan', 'grenade', 'moon', 'microphone', 'calculator']


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

data_path = "/kaggle/input/sketches-png-dataset/png"

# List of object categories
categories = os.listdir(data_path)

# How many categories and images to show
num_categories = 3
images_per_category = 3

plt.figure(figsize=(images_per_category * 3, num_categories * 3))

img_count = 1

for category in categories[:num_categories]:
    category_path = os.path.join(data_path, category)
    
    # Only continue if it's a folder
    if not os.path.isdir(category_path):
        continue
    
    images = os.listdir(category_path)[:images_per_category]

    for img_file in images:
        img_path = os.path.join(category_path, img_file)
        try:
            img = Image.open(img_path)
            plt.subplot(num_categories, images_per_category, img_count)
            plt.imshow(img, cmap='gray')
            plt.title(category)
            plt.axis('off')
            img_count += 1
        except:
            print(f"Could not open image: {img_path}")

plt.tight_layout()
plt.show()

In [54]:
import cv2
import numpy as np
import os

data_path = "/kaggle/input/sketches-png-dataset/png"
img_size = 128  # Resize all images to 128x128
X = []
y = []

categories = os.listdir(data_path)

for label, category in enumerate(categories):
    category_path = os.path.join(data_path, category)
    if not os.path.isdir(category_path):
        continue
    
    for img_file in os.listdir(category_path):
        img_path = os.path.join(category_path, img_file)
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
            img = cv2.resize(img, (img_size, img_size))        # Resize
            img = img / 255.0                                   # Normalize to [0,1]
            X.append(img)
            y.append(label)
        except:
            print(f"Failed to process: {img_path}")

X = np.array(X)
y = np.array(y)
print("hi")

hi


# Data Preprocessing

In [ ]:
import cv2
import numpy as np

# 1. Denoising
def denoise_image(image):
    return cv2.fastNlMeansDenoising((image * 255).astype(np.uint8), None, h=10, templateWindowSize=7, searchWindowSize=21) / 255.0

# 2. Contrast Enhancement using Histogram Equalization
def enhance_contrast(image):
    image_uint8 = (image * 255).astype(np.uint8)
    return cv2.equalizeHist(image_uint8) / 255.0

# 3. Apply a Filter (e.g., Sharpening)
def apply_filter(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

In [ ]:
from tqdm import tqdm
import numpy as np

X_preprocessed = []
for img in tqdm(X, desc="Preprocessing images"):
    denoised = denoise_image(img)
    enhanced = enhance_contrast(denoised)
    filtered = apply_filter(enhanced)
    X_preprocessed.append(filtered)

X_preprocessed = np.array(X_preprocessed)


Visualising pre-processed data

In [ ]:
import matplotlib.pyplot as plt

# Display first 5 images from the preprocessed list
plt.figure(figsize=(12, 4))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_preprocessed[i], cmap='gray')
    plt.title(f'Image {i+1}')
    plt.axis('off')

plt.tight_layout()
plt.show()


# Feature Extraction

In [ ]:
!pip install scikit-image

HOG Features

In [ ]:
from skimage.feature import hog

# Extract HOG features from preprocessed images
hog_features = []

# Parameters: adjust based on dataset complexity and image size
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys'
}

for img in X_preprocessed:
    features = hog(img, **hog_params)
    hog_features.append(features)

hog_features = np.array(hog_features)
print("HOG feature shape:", hog_features.shape)

CNN Features

In [ ]:
!pip install torch torchvision
print("!")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm  # Progress bar

# Define the transform: resize to 224x224 and normalize for ImageNet
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

print("123")

# Load pretrained ResNet-34 model
resnet = models.resnet34(pretrained=True)
resnet.eval()
print("123")


# Remove the final fully connected layer to get features
resnet_feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Output shape: (batch, 512, 1, 1)

print("123")

# Convert preprocessed images to CNN features
cnn_features = []

for img in tqdm(X_preprocessed, desc="Extracting CNN features"):
    # Ensure image is in uint8 format (needed for PIL)
    img_tensor = transform((img * 255).astype(np.uint8))
    img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        feat = resnet_feature_extractor(img_tensor)
        feat = feat.view(-1).numpy()  # Flatten to 1D feature vector
        cnn_features.append(feat)

# Convert list to numpy array
cnn_features = np.array(cnn_features)
print("CNN feature shape:", cnn_features.shape)

LBP Features

In [ ]:
from skimage.feature import local_binary_pattern
from tqdm import tqdm

# Parameters for LBP
radius = 1
n_points = 8 * radius
method = 'uniform'  # options: 'default', 'ror', 'uniform', 'var'

def compute_lbp_fast(img):
    lbp = local_binary_pattern(img, n_points, radius, method)
    # Use histogram to reduce feature size
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, n_points + 3),
                             range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize histogram
    return hist

# Apply to all images
lbp_features_fast = np.array([compute_lbp_fast((img * 255).astype(np.uint8)) for img in tqdm(X_preprocessed, desc="Fast LBP")])
print("Fast LBP feature shape:", lbp_features_fast.shape)

Saving Features as Pickle files

In [ ]:
import pickle

# Save HOG features
with open("hog_features.pkl", "wb") as f:
    pickle.dump(hog_features, f)

# Save CNN features
with open("cnn_features.pkl", "wb") as f:
    pickle.dump(cnn_features, f)

# Save LBP features
with open("lbp_features.pkl", "wb") as f:
    pickle.dump(lbp_features_fast, f)

print("All feature files saved successfully!")

Loading features from pickle files

In [55]:
import pickle
import numpy as np

# Path to your uploaded dataset
base_path = "/kaggle/input/sketch-features/"

# Load HOG features
with open(base_path + "hog_features.pkl", "rb") as f:
    hog_features = pickle.load(f)

# Load LBP features
with open(base_path + "lbp_features.pkl", "rb") as f:
    lbp_features = pickle.load(f)

# Load CNN features
with open(base_path + "cnn_features.pkl", "rb") as f:
    cnn_features = pickle.load(f)

print("Shapes:")
print("HOG:", hog_features.shape)
print("LBP:", lbp_features.shape)
print("CNN:", cnn_features.shape)

Shapes:
HOG: (20000, 8100)
LBP: (20000, 10)
CNN: (20000, 512)
Combined feature shape: (20000, 522)


Standardizing features and combining them 

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize the features separately
scaler_hog = StandardScaler()
scaler_lbp = StandardScaler()
scaler_cnn = StandardScaler()

hog_scaled = scaler_hog.fit_transform(hog_features)
lbp_scaled = scaler_lbp.fit_transform(lbp_features)
cnn_scaled = scaler_cnn.fit_transform(cnn_features)

# Combine all features
combined_features = np.concatenate([lbp_scaled, cnn_scaled], axis=1)
print("Combined feature shape:", combined_features.shape)

In [57]:
class_labels = sorted(os.listdir(data_path))

# PCA

In [58]:
from sklearn.decomposition import PCA
import joblib
from sklearn.model_selection import train_test_split
# Apply PCA to retain 95% variance
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(combined_features)

# Split and re-train (example with Logistic Regression)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Model Implementation with PCA

KNN

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_pca, y_train_pca)

# Predict and evaluate
y_pred = knn.predict(X_test_pca)

print("KNN Accuracy:", accuracy_score(y_test_pca, y_pred))

# import joblib
# joblib.dump(knn_pca, 'knn_model_pca.pkl')


KNN Accuracy: 0.2685


Random Forest

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_pca, y_train_pca)

# Fast batch prediction
y_pred_rf = rf_clf.predict(X_test_pca)

# Evaluate accuracy
print("Random Forest Accuracy (Combined Features):", accuracy_score(y_test_pca, y_pred_rf))
# joblib.dump(rf_clf, 'random_forest_model.pkl')
# import joblib

Random Forest Accuracy (Combined Features): 0.27025


Logistic Regression

In [61]:
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# # Split
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train_pca, y_train_pca)

# Predict with tqdm
y_pred = []
for sample in tqdm(X_test_pca, desc="Predicting with Logistic Regression"):
    pred = lr_clf.predict(sample.reshape(1, -1))
    y_pred.append(pred[0])

# Accuracy
print("Logistic Regression Accuracy (Combined):", accuracy_score(y_test_pca, y_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Predicting with Logistic Regression: 100%|██████████| 4000/4000 [00:00<00:00, 9105.89it/s] 

Logistic Regression Accuracy (Combined): 0.44075


SVM

In [62]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Train the SVM classifier (RBF kernel by default)
svm_clf = SVC(kernel='rbf', probability=False)  # You can also try 'linear' or 'poly'
svm_clf.fit(X_train_pca, y_train_pca)

# Predict with tqdm progress bar
y_pred_svm = []
for sample in tqdm(X_test_pca, desc="Predicting with SVM", unit="sample"):
    pred = svm_clf.predict(sample.reshape(1, -1))
    y_pred_svm.append(pred[0])

# Evaluate accuracy
print("SVM Accuracy (Combined Features):", accuracy_score(y_test_pca, y_pred_svm))

Predicting with SVM: 100%|██████████| 4000/4000 [00:46<00:00, 85.84sample/s]

SVM Accuracy (Combined Features): 0.432


MLP

In [63]:
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm

mlp = MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=1, warm_start=True, random_state=42)

n_epochs = 100
for _ in tqdm(range(n_epochs), desc="Training MLP"):
    mlp.fit(X_train_pca, y_train_pca)

y_pred_mlp = mlp.predict(X_test_pca)
print("MLP Accuracy:", accuracy_score(y_test_pca, y_pred_mlp))

Training MLP:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
Training MLP: 100%|██████████| 100/100 [00:49<00:00,  2.02it/s]

MLP Accuracy: 0.4085


Naive Bayes

In [64]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Initialize Naive Bayes model
nb_clf = GaussianNB()

# Split
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train the model
nb_clf.fit(X_train_pca, y_train_pca)

# Predict with tqdm progress bar
y_pred_nb = []
for sample in tqdm(X_test_pca, desc="Predicting with Naive Bayes", unit="sample"):
    pred = nb_clf.predict(sample.reshape(1, -1))
    y_pred_nb.append(pred[0])

# Evaluate accuracy
print("Naive Bayes Accuracy (Combined Features):", accuracy_score(y_test_pca, y_pred_nb))

Predicting with Naive Bayes: 100%|██████████| 4000/4000 [00:22<00:00, 180.04sample/s]

Naive Bayes Accuracy (Combined Features): 0.3465


# Model Implementation without PCA

Logistic Regression

In [65]:
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Split
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict with tqdm
y_pred_lr = []
for sample in tqdm(X_test, desc="Predicting with Logistic Regression"):
    pred = clf.predict(sample.reshape(1, -1))
    y_pred_lr.append(pred[0])

# Accuracy
print("Logistic Regression Accuracy (Combined):", accuracy_score(y_test, y_pred_lr))

Predicting with Logistic Regression: 100%|██████████| 4000/4000 [00:00<00:00, 7926.48it/s]

Logistic Regression Accuracy (Combined): 0.47675


SVM

In [66]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Train the SVM classifier (RBF kernel by default)
svm_clf_p = SVC(kernel='rbf', probability=False)  # You can also try 'linear' or 'poly'
svm_clf_p.fit(X_train, y_train)

# Predict with tqdm progress bar
y_pred_svm = []
for sample in tqdm(X_test, desc="Predicting with SVM", unit="sample"):
    pred = svm_clf_p.predict(sample.reshape(1, -1))
    y_pred_svm.append(pred[0])

# Evaluate accuracy
print("SVM Accuracy (Combined Features):", accuracy_score(y_test, y_pred_svm))

Predicting with SVM: 100%|██████████| 4000/4000 [01:02<00:00, 64.45sample/s]

SVM Accuracy (Combined Features): 0.4365


Random Forest

In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train the Random Forest classifier
rf_clf_p = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf_p.fit(X_train, y_train)

# Fast batch prediction
y_pred_rf = rf_clf_p.predict(X_test)

# Evaluate accuracy
print("Random Forest Accuracy (Combined Features):", accuracy_score(y_test, y_pred_rf))
# joblib.dump(rf_clf, 'random_forest_model.pkl')
# import joblib

Random Forest Accuracy (Combined Features): 0.32175


KNN

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the data
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train KNN
knn_p = KNeighborsClassifier(n_neighbors=3)
knn_p.fit(X_train, y_train)

# Predict and evaluate
y_pred = knn_p.predict(X_test)

print("KNN Accuracy:", accuracy_score(y_test, y_pred))

# import joblib
# joblib.dump(knn_pca, 'knn_model_pca.pkl')

MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm

mlp_p = MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=1, warm_start=True, random_state=42)

n_epochs = 100
for _ in tqdm(range(n_epochs), desc="Training MLP"):
    mlp_p.fit(X_train, y_train)

y_pred_mlp = mlp_p.predict(X_test)
print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Initialize Naive Bayes model
nb_clf_p = GaussianNB()

# Split
# X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Train the model
nb_clf_p.fit(X_train, y_train)

# Predict with tqdm progress bar
y_pred_nb = []
for sample in tqdm(X_test, desc="Predicting with Naive Bayes", unit="sample"):
    pred = nb_clf_p.predict(sample.reshape(1, -1))
    y_pred_nb.append(pred[0])

# Evaluate accuracy
print("Naive Bayes Accuracy (Combined Features):", accuracy_score(y_test, y_pred_nb))

# TSNE visualisation of features

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Reduce dimensionality for visualization
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
features_2d = tsne.fit_transform(combined_features)

# Plot
plt.figure(figsize=(10, 6))
scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], c=y, cmap='tab10', s=10, alpha=0.7)
plt.title("t-SNE Visualization of Combined Features")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(scatter, label="Class Label")
plt.grid(True)
plt.show()

# Accuracy Plot

In [ ]:
import matplotlib.pyplot as plt

# Store accuracies
accuracies = {
    "KNN": accuracy_score(y_test, knn_p.predict(X_test)),
    "Random Forest": accuracy_score(y_test, rf_clf_p.predict(X_test)),
    "LogReg": accuracy_score(y_test, y_pred_lr),
    "MLP": accuracy_score(y_test, y_pred_mlp),
    "Naive Bayes": accuracy_score(y_test_pca, y_pred_nb),
    "SVM": accuracy_score(y_test, y_pred_svm),
}

# Plot
plt.figure(figsize=(8, 5))
plt.bar(accuracies.keys(), accuracies.values(), color=['skyblue', 'lightgreen', 'salmon'])
plt.ylabel("Accuracy")
plt.title("Model Accuracy Comparison")
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


# Precision, Recall, F1 score Plot

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = {
    "KNN": precision_score(y_test, knn_p.predict(X_test), average='macro'),
    "Random Forest": precision_score(y_test, rf_clf_p.predict(X_test), average='macro'),
    "LogReg": precision_score(y_test, y_pred_lr, average='macro'),
    "MLP": precision_score(y_test, y_pred_mlp, average='macro'),
    "Naive Bayes": precision_score(y_test_pca, y_pred_nb, average='macro'),
    "SVM": precision_score(y_test, y_pred_svm, average='macro'),
}

recall = {
    "KNN": recall_score(y_test, knn_p.predict(X_test), average='macro'),
    "Random Forest": recall_score(y_test, rf_clf_p.predict(X_test), average='macro'),
    "LogReg": recall_score(y_test, y_pred_lr, average='macro'),
    "MLP": recall_score(y_test, y_pred_mlp, average='macro'),
    "Naive Bayes": recall_score(y_test_pca, y_pred_nb, average='macro'),
    "SVM": recall_score(y_test, y_pred_svm, average='macro'),
}

f1 = {
    "KNN": f1_score(y_test, knn_p.predict(X_test), average='macro'),
    "Random Forest": f1_score(y_test, rf_clf_p.predict(X_test), average='macro'),
    "LogReg": f1_score(y_test, y_pred_lr, average='macro'),
    "MLP": f1_score(y_test, y_pred_mlp, average='macro'),
    "Naive Bayes": f1_score(y_test_pca, y_pred_nb, average='macro'),
    "SVM": f1_score(y_test, y_pred_svm, average='macro'),
}
print("done")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# X-axis: model names
models = list(precision.keys())

# Y-axis values
precision_scores = [precision[m] for m in models]
recall_scores = [recall[m] for m in models]
f1_scores = [f1[m] for m in models]

# Positioning
x = np.arange(len(models))
width = 0.25

# Plot
plt.figure(figsize=(12, 6))
plt.bar(x - width, precision_scores, width, label='Precision', color='skyblue')
plt.bar(x, recall_scores, width, label='Recall', color='orange')
plt.bar(x + width, f1_scores, width, label='F1 Score', color='green')

# Labels
plt.xticks(x, models)
plt.ylabel("Score")
plt.ylim(0, 1)
plt.title("Precision, Recall, and F1 Score Comparison")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


# Saving Models as Pickle files

In [ ]:
import joblib

joblib.dump(knn, 'knn_model.pkl')
joblib.dump(clf, 'logistic_regression_model.pkl')
joblib.dump(rf_clf, 'random_forest_model.pkl')
joblib.dump(svm_clf, 'svm_model.pkl')
joblib.dump(nb_clf, 'naive_bayes_model.pkl')
joblib.dump(mlp_clf, 'mlp_model.pkl')